In [ ]:
from sagemaker.profiler import ProfilerConfig 

profiler_config = ProfilerConfig(
    profiling_interval_millis=500,
    profiling_parameters={
        "ProfilerEnabled": str(True),
        "UsePyinstrument": str(True),
        "DetailedProfilingConfig": "{\"StartStep\": 2, \"NumSteps\": 2}"
    }
)

In [ ]:
!pip show sagemaker

In [ ]:
import boto3

session = boto3.session.Session()
region = session.region_name

#image_name = f'385479125792.dkr.ecr.{region}.amazonaws.com/profiler-gpu:pytorch_0813'
image_name = f'072677473360.dkr.ecr.us-east-1.amazonaws.com/smprofiler-pytorch-gpu:1.5.1_aws_dataloader'

hyperparameters = {"batch_size":2048, "gpu":True, "pin_memory":True, "workers":4, "epoch":15, "model":"resnext50_32x4d"}
hyperparameters = {"batch_size":2048, "gpu":True, "pin_memory":True, "workers":4, "epoch":15, "model":"resnext101_32x8d"}
hyperparameters = {"batch_size":2048, "gpu":True, "pin_memory":True, "workers":0, "epoch":5, "model":"resnext101_32x8d"}
hyperparameters = {"batch_size":2048, "gpu":True, "pin_memory":False, "workers":0, "epoch":5, "model":"resnext101_32x8d"}
entry_point_script = 'pytorch_res50_cifar10_dataloader.py'
base_job_name="pt-singleworker-no-pinmemoryresnext101"
instance_count = 1

# Uncomment for horovod based multigpu training
#hyperparameters = {"script":"pt_res50_cifar10_horovod_dataloader.py", "model":"resnext101_32x8d", "batch_size":2048, "epoch":5}
#entry_point_script = 'horovod_test_launcher.py'
#base_job_name = "pt-horvod-multigpu-resnet101"

# Uncomment for distributed API based training
#hyperparameters = {"training_script":"pt_res50_cifar10_distributed.py", "nproc_per_node":4, "nnodes":2}
#entry_point_script = 'distributed_launch.py'
#instance_count = 2
#base_job_name = "pt-distributed=multiinstance"

# Uncomment for running the training with custom dataset.
#hyperparameters = {"batch_size":128, "gpu":True, "pin_memory":True, "workers":4, "epoch":1, "data_folder":"/opt/ml/input/data/training"}
#base_job_name = 'pt-histopathologic'
#entry_script = "histopathologic_training.py"

print(f"image being used is {image_name}")

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    role=sagemaker.get_execution_role(),
    image_name=image_name,
    train_instance_count=instance_count,
    train_instance_type='ml.p3.8xlarge',
    source_dir='pytorch/scripts',
    entry_point=entry_point_script,
    framework_version='1.5.1',
    hyperparameters=hyperparameters,
    profiler_config=profiler_config,
    base_job_name=base_job_name)

In [ ]:
estimator.fit(wait=False)